In [9]:
from functools import reduce
import gc
import numpy as np

"""STVM 계산(3차로 점선) + 지상부 추가"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 5400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6

# 본선부 검지기
main_line = 60

# 지상부_진출 검지기
exit_line = 265
############################


######## 램프 ###############
# 유입램프
input_ramp = 59

# 유출램프
output_ramp = 61
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 평균속도
def speed_mean(df):
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도
def density_mean(df):
    density_mean_df = df.assign(K = df["V_count"] * 12 / df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 가중 중차량 비율
def heavy_rate(original_df):

    # 진입부 검지기 선정
    measurement = enter_line

    heavy_df = (
        original_df[original_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        original_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup", "New_Measurement"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 진입 포화도
def entry_saturation(original_df):
    # 실측용량 C
    max_capacity = 6600
    entry_saturation_df = (
        # 진입부에서 유입된 교통량이므로 진입부 중 한 개의 검지기를 선정하여 측정
        original_df[original_df["New_Measurement"] == enter_line]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df


# 램프 유출입 비율
def rfr_rate(original_df):
    """
    1. 본선 1개의 검지기 data 수집(New_Measurement = 60)
    2. 위의 검지기 앞, 뒤 검지기 data (59 / 61)
    3. RFR 연산
    """
    main_df = (
        original_df[original_df["New_Measurement"] == main_line] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == input_ramp] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == output_ramp] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0.2값으로 fix
    main_df["RFR"] = 0.2
    return main_df

# 유출 정상성 비율
def output_normality(original_df):
    # 6번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    entry_df = original_df[original_df["New_Measurement"] == enter_line][["VehNo", "t(Entry)"]]

    # 265번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    exit_df = original_df[original_df["New_Measurement"] == exit_line][["VehNo", "t(Entry)"]]

    # 차량 번호로 그룹화 후 시간의 최솟값(중복제거)
    entry_first = (
        entry_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()  # Series → DataFrame
        .rename(columns={"t(Entry)": "t_entry"})
    )
    exit_first = (
        exit_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()
        .rename(columns={"t(Entry)": "t_exit"})
    )

    # 지연시간
    merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
    merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
    merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

    # 지연시간(중앙값) → lag_bins
    if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])): # delay_sec의 값이 유효하면
        lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / 300)) # 단위시간으로 나눴을 때의 중간값 => 3(900초) => 진입한 차량이 진출을 통과하는데 평균 900초가 걸림
    else:
        lag_bins = 0  # 데이터 부족 시 동시간 매칭

    # TimeGroup별 진입/유출 카운트 집계
    entry_count = (original_df[original_df["New_Measurement"] == enter_line]
                .groupby("TimeGroup").size().reset_index(name="Q_in"))
    exit_count  = (original_df[original_df["New_Measurement"] == exit_line]
                .groupby("TimeGroup").size().reset_index(name="Q_out"))

    merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")

    # Q_out을 지연 시간만큼 shift
    merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)

    # 1−F(outrate) 계산
    merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)
    merged_counts["1-F(outrate)"] = 1 - merged_counts["F(outrate)"]
    return merged_counts


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):


    # TimeGroup 기준으로  Merge
    merged_df = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "New_Measurement", "rate"]], on=["TimeGroup", "New_Measurement"])
    .merge(entry_saturation_df[["TimeGroup", "Phi_진입"]], on="TimeGroup")
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "1-F(outrate)"]], on="TimeGroup")
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["1-F(outrate)"]
    )

    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    # 지상부 1~5, 276~280 제거
    meodify_df = df.copy()
    remove_values = list(range(1, 6)) + list(range(276, 281))

    meodify_df = meodify_df[~meodify_df["New_Measurement"].isin(remove_values)]
    meodify_df["New_Measurement"] = meodify_df["New_Measurement"].apply(
        lambda x: -5 + (x - 6) if 6 <= x <= 10 else (x - 10 if x >= 11 else x)
    )
    return meodify_df

def calculate_avg(df):

    def avg_range(df, low, high):
        cols = [c for c in df.columns if low <= c <= high]
        return df[cols].mean(axis=1) if cols else np.nan

    avg_df = pd.DataFrame(index=df.index)
    avg_df["지상부_진입"] = avg_range(df, -5, -1)
    avg_df["진입부"] = avg_range(df, 1, 30)
    avg_df["본선부"] = avg_range(df, 31, 230)
    avg_df["진출부"] = avg_range(df, 231, 260)
    avg_df["지상부_진출"] = avg_range(df, 261, 265)

    return avg_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "1-F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def pivot_stvm(stvm_df):
    pivot_df = stvm_df.pivot(index="TimeGroup", columns="New_Measurement", values="STVM")
    return pivot_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K", ]]
    return avg_df

def pivot_delta_speed(speed_df):
    copy_df = speed_df.copy()
    delta_speed_pivot_df = modify_frame(copy_df)
    delta_speed_pivot_df = pd.pivot(delta_speed_pivot_df, index="TimeGroup", columns= "New_Measurement", values="delta_V")
    return delta_speed_pivot_df

def pivot_delta_density(density_df):
    copy_df = density_df.copy()
    delta_denisy_pivot_df = modify_frame(copy_df)
    delta_denisy_pivot_df = pd.pivot(delta_denisy_pivot_df, index="TimeGroup", columns= "New_Measurement", values="delta_K")
    return delta_denisy_pivot_df

def pivot_speed(speed_df) :
    display("speed_df : ", speed_df)
    copy_df = speed_df.copy()
    speed_pivot_df = modify_frame(copy_df)
    speed_pivot_df = pd.pivot(speed_pivot_df, index="TimeGroup", columns= "New_Measurement", values="V_mean")
    display("speed_pivot_df : ", speed_pivot_df)
    return speed_pivot_df

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")


def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = r"C:\VISSIM_Workspace\mer파일\1750"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()

for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 평균속도
                speed_df = speed_mean(original_df)

                # 밀도
                density_df = density_mean(speed_df)

                # 가중 중차량 비율
                heavy_df = heavy_rate(original_df)

                # 진입 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 유출입 비율
                rfr_df = rfr_rate(original_df)

                # 진출 정상성
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)


                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                #save_to_excel(z_score_df, folder_path, "환산점수", i)

                # 변수별 TimeGroup별 평균
                variable_time_df = variable_timegroup_avg(stvm_df)
                #save_to_excel(variable_time_df, folder_path, "변수별 평균", i)

                # 변수별 전체 평균
                variable_total_df = variable_total_avg(variable_time_df)
                #save_to_excel(variable_total_df, folder_path, "변수별 총 평균", i)

                # STVM 피봇
                stvm_pivot_df = pivot_stvm(stvm_df)
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도, 밀도 전체 평균값
                speed_density_df = speed_density_avg(density_df)
                #save_to_excel(speed_density_df, folder_path, "속도_밀도 평균값", i)

                # 속도 변화 피봇
                delta_speed_pivot_df = pivot_delta_speed(speed_df)
                #save_to_excel(delta_speed_pivot_df, folder_path, "속도변화량", i)

                # 속도값 피봇
                speed_pivot_df = pivot_speed(speed_df)
                save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 밀도 변화 피봇
                delta_density_pivot_df = pivot_delta_density(density_df)
                #save_to_excel(delta_density_pivot_df, folder_path, "밀도변화량", i)

                # 메모리 정리
                del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()

'speed_df : '

,TimeGroup,New_Measurement,V_mean,V_count,V_next,delta_V
0,1800~2100,1,102.390268,411,103.249034,0.008387
1,1800~2100,2,103.249034,414,103.592754,0.003329
2,1800~2100,3,103.592754,414,103.448077,-0.001397
3,1800~2100,4,103.448077,416,103.066106,-0.003692
4,1800~2100,5,103.066106,416,102.921792,-0.001400
...,...,...,...,...,...,...
3355,5100~5400,276,99.528788,462,99.098715,-0.004321
3356,5100~5400,277,99.098715,467,98.781992,-0.003196
3357,5100~5400,278,98.781992,472,98.738608,-0.000439
3358,5100~5400,279,98.738608,474,101.152452,0.024447


'speed_pivot_df : '

New_Measurement,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,102.921792,102.888293,102.820874,102.830769,102.088702,102.301716,102.406634,102.318812,102.461097,102.390148,...,97.223737,97.753197,97.945736,98.058140,98.067959,98.855754,99.737532,99.758355,99.517571,99.333592
2100~2400,102.038444,102.211982,101.901392,101.368000,101.026230,100.814286,100.568807,100.360321,100.813242,100.738673,...,97.189189,96.600000,95.649896,95.579303,96.055031,97.690456,99.083644,99.622107,99.041580,98.668201
2400~2700,101.486239,100.855809,99.940816,99.928733,100.289703,100.029263,99.601860,99.380000,99.293995,99.142431,...,96.571236,96.383632,96.168764,95.917273,96.007466,97.735520,98.995506,98.807848,98.226726,98.029712
2700~3000,102.322927,101.768719,101.493086,101.068293,100.552657,99.769324,99.711990,99.851559,99.717676,99.569100,...,96.461995,96.743705,97.176706,97.479625,96.957674,97.273103,98.141876,98.732407,98.342361,98.204408
3000~3300,101.786062,101.653863,101.401774,100.908241,100.903118,101.339062,101.605405,101.622297,101.413933,101.646697,...,98.549370,98.516332,98.695431,98.914937,98.813033,99.551128,100.751010,101.144584,101.596410,101.522423
3300~3600,103.156098,102.944203,102.671463,102.406235,102.284504,101.875303,101.841429,101.605251,101.378897,101.256840,...,97.597032,97.800231,97.809217,97.665589,97.548598,98.934507,99.839387,100.082547,100.078824,99.894419
3600~3900,100.746224,100.996520,101.018925,101.360095,101.297619,101.259091,101.054589,100.856325,100.420706,100.521780,...,94.401617,94.113801,93.977303,94.244912,95.064018,96.098222,96.860177,97.735333,98.130263,98.078634
3900~4200,102.873581,102.298698,102.007158,101.928908,101.733049,101.243923,101.075424,101.187473,101.052331,100.835776,...,96.261521,95.962212,95.638303,95.625459,95.530752,97.011791,98.308904,99.221918,99.005081,98.597235
4200~4500,102.080044,102.235385,102.334061,101.916228,101.608791,101.751982,101.905556,101.993065,101.888235,101.645393,...,96.459821,96.899333,97.465778,97.401124,97.532584,98.581292,99.061504,99.741703,100.275599,99.966157


속도값_1.xlsx 생성 완료


'speed_df : '

,TimeGroup,New_Measurement,V_mean,V_count,V_next,delta_V
0,1800~2100,1,99.688384,396,100.418296,0.007322
1,1800~2100,2,100.418296,399,100.715152,0.002956
2,1800~2100,3,100.715152,396,100.167254,-0.005440
3,1800~2100,4,100.167254,397,99.909343,-0.002575
4,1800~2100,5,99.909343,396,99.987949,0.000787
...,...,...,...,...,...,...
3355,5100~5400,276,96.818750,368,96.942204,0.001275
3356,5100~5400,277,96.942204,372,97.141019,0.002051
3357,5100~5400,278,97.141019,373,97.840710,0.007203
3358,5100~5400,279,97.840710,366,99.225348,0.014152


'speed_pivot_df : '

New_Measurement,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,99.987949,99.432995,98.830886,98.267259,97.918701,97.641667,97.597179,98.004922,98.251436,98.441602,...,51.857584,50.070604,46.704545,50.179843,52.855145,71.941689,83.393684,89.318277,91.809351,94.188052
2100~2400,98.676000,98.168957,97.703580,97.245585,96.975529,97.077123,97.029665,96.755660,96.378014,96.131415,...,50.511002,50.037226,50.016425,49.473529,53.068719,71.796341,82.557561,89.098020,92.334739,94.584788
2400~2700,96.634048,96.281687,96.268182,96.218377,95.876850,95.539858,95.508491,95.776485,95.975529,96.004450,...,48.335154,48.305596,46.600993,47.186667,50.659900,70.568687,81.867176,88.177041,91.548711,93.934271
2700~3000,98.993703,98.696526,98.082555,97.757921,97.473067,97.023858,96.481218,96.285427,96.039798,95.745000,...,48.704358,49.416972,49.763869,49.835849,50.876888,70.252596,82.047404,89.021205,91.889845,94.934967
3000~3300,100.402410,100.098301,99.896798,99.762287,99.022892,97.323628,96.478404,96.620665,97.446860,97.457353,...,46.506010,45.639855,46.417308,49.637470,51.123587,70.870000,82.560100,89.654937,91.999746,94.252764
3300~3600,98.468998,98.551852,97.994919,97.719169,97.830254,98.194432,97.987617,96.997917,96.785057,96.909589,...,49.765499,48.852139,46.795745,49.853495,50.752000,70.626933,82.229600,89.030871,92.205121,94.239837
3600~3900,101.368649,101.308696,101.051093,100.708470,100.549591,100.741463,101.293207,100.887978,100.677112,100.622283,...,50.137685,47.693120,47.593382,49.579562,49.148910,69.750718,81.820096,88.592584,90.836000,93.026107
3900~4200,98.243326,97.759695,97.264924,96.880088,96.962996,97.071965,96.891353,96.732230,96.729540,97.011013,...,46.614925,46.697425,48.363283,47.271522,48.080879,69.647894,81.534812,88.247556,90.851011,92.694582
4200~4500,100.113626,99.790487,99.657373,99.635402,99.415349,98.942725,99.103661,98.809217,98.419347,98.284615,...,42.230120,43.087681,46.440191,47.691765,49.458945,69.796380,81.483032,89.491629,92.624270,94.885649


속도값_2.xlsx 생성 완료
